# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

#### Answer
---
This data represent 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



#### Answer
---
By using `df.info()` it was observed that there are no missing values in this dataset; however, there are some "unknown" values that might need to be treated as missing values. All columns of type object will need to be encoded.

In [4]:
# Get the unique values for each feature of type object
object_columns = df.select_dtypes(include="object")
unique_values = {}
for col in object_columns:
    print(f'Column: {col}')
    print(object_columns[col].unique())

Column: job
['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
Column: marital
['married' 'single' 'divorced' 'unknown']
Column: education
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
Column: default
['no' 'unknown' 'yes']
Column: housing
['no' 'yes' 'unknown']
Column: loan
['no' 'yes' 'unknown']
Column: contact
['telephone' 'cellular']
Column: month
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
Column: day_of_week
['mon' 'tue' 'wed' 'thu' 'fri']
Column: poutcome
['nonexistent' 'failure' 'success']
Column: y
['no' 'yes']


In [5]:
# Get the value counts for each feature
for col in df.columns:
    print(f'Column: {col}')
    print(df[col].value_counts())

Column: age
age
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
94       1
87       1
95       1
Name: count, Length: 78, dtype: int64
Column: job
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64
Column: marital
marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64
Column: education
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64
Column: default
default
no         32588
unknown     8597
yes            3
Name: count, dtype: int6

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

#### Business Objective
---
To explore different models and select the one that best predicts whether a customer will commit to a long-term deposit subscription to maximize return of investment of marketing efforts. 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [7]:
# Remove 'unknown' values
df = df.replace('unknown', np.nan)
no_missing_values_df = df.dropna()

In [8]:
# Select only bank features to train the models
bank_features = ['age','job', 'marital', 'education', 'default', 'housing', 'loan']
X = no_missing_values_df[bank_features]
y = no_missing_values_df['y']

In [9]:
# Define a selector to get only the nominal features (of type object)
selector = make_column_selector(dtype_include=object)

In [10]:
# Encode nominal features
transformer = make_column_transformer((OneHotEncoder(drop = 'first'), selector),                                      
                                     remainder = StandardScaler())

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42,
                                                   stratify = y)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [12]:
baseline = y_test.value_counts(normalize = True)
baseline

y
no     0.873393
yes    0.126607
Name: proportion, dtype: float64

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [13]:
start_time = time.time()
lgr_pipe = Pipeline([
    ('transformer', transformer),
    ('lgr', LogisticRegression(random_state=42, max_iter = 1000))
])
lgr_pipe.fit(X_train, y_train)
lgr_time = time.time() - start_time

### Problem 9: Score the Model

What is the accuracy of your model?

In [14]:
lgr_acc_train = lgr_pipe.score(X_train, y_train)
lgr_acc_train

0.873436543339456

In [15]:
lgr_acc_test = lgr_pipe.score(X_test, y_test)
lgr_acc_test

0.8733928102860141

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

#### KNeighborsClassifier
---

In [16]:
knn_transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), selector),
                                     remainder = StandardScaler())
start_time = time.time()
knn_pipe = Pipeline([
    ('transform', knn_transformer), 
    ('knn', KNeighborsClassifier())
    ])
knn_pipe.fit(X_train, y_train)
knn_time = time.time() - start_time

In [17]:
knn_score_train = knn_pipe.score(X_train, y_train)
knn_score_train

0.8772413189888918

In [18]:
knn_score_test = knn_pipe.score(X_test, y_test)
knn_score_test

0.8604040934138022

#### DecisionTreeClassifier
---

In [19]:
dtree_transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), selector),                                      
                                     remainder = 'passthrough')
start_time = time.time()
dtree_pipe = Pipeline([
    ('transformer', dtree_transformer),
    ('dtree', DecisionTreeClassifier())
])
dtree_pipe.fit(X_test, y_test)
dtree_time = time.time() - start_time

In [20]:
dtree_score_train = dtree_pipe.score(X_train, y_train)
dtree_score_train

0.8360447826467244

In [21]:
dtree_score_test = dtree_pipe.score(X_test, y_test)
dtree_score_test

0.9193125163998951

#### Suport Vector Machine (SVM)
---

In [22]:
svc_transformer = make_column_transformer((OneHotEncoder(drop='if_binary'), selector),
                                          remainder = StandardScaler())
start_time = time.time()
svc_pipe = Pipeline([
    ('transformer', svc_transformer),
    ('svc', SVC())
])
svc_pipe = svc_pipe.fit(X_train, y_train)
svc_time = time.time() - start_time

In [23]:
svc_score_train = svc_pipe.score(X_train, y_train)
svc_score_train

0.8741362721945246

In [24]:
svc_score_test = svc_pipe.score(X_test, y_test)
svc_score_test

0.8728680136447127

#### Performance Comparison
---

In [25]:
comparisons = [
    ['LogisticRegression',lgr_time,lgr_acc_train,lgr_acc_test],
    ['KNeighborsClassifier',knn_time,knn_score_train,knn_score_test],
    ['DecisionTreeClassifier',dtree_time,dtree_score_train,dtree_score_test],
    ['SVC',svc_time,svc_score_train,svc_score_test]
]
pd.DataFrame(comparisons, columns=['Model', 'Train Time', 'Train Accuracy', 'Test Accuracy'])

,Model,Train Time,Train Accuracy,Test Accuracy
0,LogisticRegression,0.070875,0.873437,0.873393
1,KNeighborsClassifier,0.046308,0.877241,0.860404
2,DecisionTreeClassifier,0.029835,0.836045,0.919313
3,SVC,9.723132,0.874136,0.872868


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions